<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/073_Clean_Tool_Dependency_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is about **clean, automatic dependency injection** for tools in AI agent systems, using the `ActionContext` pattern in conjunction with an `Environment`.

Here’s what you should be focusing on:

Understand how to **decouple tools from the agent orchestration logic** by allowing the environment to handle dependency injection — making tools easier to maintain, test, and secure.

---

## 📘 Clean Tool Dependency Injection (Markdown Summary)

### 🔧 Why Clean Dependency Injection Matters

When designing tools for AI agents:

* Some tools need **access to context** (e.g. memory, authentication tokens).
* Others (like a simple string formatter) don’t need any external dependencies.

Rather than forcing **every tool to accept the same parameters**, we want to:

* Inject **only the necessary dependencies**
* **Avoid tightly coupling tools** to the agent logic
* Keep the agent orchestration **simple and focused**

---

### 🔥 Bad Example: Manual Dependency Management (What NOT to do)

```python
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    action_def, action = self.get_action(response)
    args = action["args"].copy()

    # Manually check and inject dependencies
    if needs_action_context(action_def):
        args["action_context"] = action_context
    if needs_auth_token(action_def):
        args["_auth_token"] = action_context.get("auth_token")
    
    result = action_def.execute(**args)
    return result
```

❌ Problems:

* Bloats agent logic
* Breaks separation of concerns
* Risk of **over-sharing sensitive dependencies**
* Harder to test and maintain

---

### ✅ Clean Approach: Delegate to the `Environment`

```python
def handle_agent_response(self, action_context: ActionContext, response: str) -> dict:
    action_def, action = self.get_action(response)
    return self.environment.execute_action(self, action_context, action_def, action["args"])
```

✅ The agent just says **what** to do — the environment handles **how**.

---

### 🧱 The Environment Handles Dependency Injection

```python
class PythonEnvironment(Environment):
    def execute_action(self, agent, action_context: ActionContext, action: Action, args: dict) -> dict:
        args_copy = args.copy()

        # Inject action_context if needed
        if has_named_parameter(action.function, "action_context"):
            args_copy["action_context"] = action_context

        # Inject specific properties from context using _prefix convention
        for key, value in action_context.properties.items():
            param_name = "_" + key
            if has_named_parameter(action.function, param_name):
                args_copy[param_name] = value

        result = action.execute(**args_copy)
        return self.format_result(result)
```

### 📦 Tool Interface Conventions

* If a tool defines a parameter `action_context`, it receives the full context.
* If a tool defines a parameter like `_auth_token`, the environment will look for `auth_token` in the `ActionContext` and pass it in automatically.

---

### 🧼 Example: Clean, Minimal Tool

```python
@register_tool(description="Convert text to uppercase")
def to_uppercase(text: str) -> str:
    return text.upper()
```

### 🔐 Example: Authenticated Tool

```python
@register_tool(description="Update user profile")
def update_profile(action_context: ActionContext, username: str, _auth_token: str) -> dict:
    return make_authenticated_request(_auth_token, username)
```

Only tools that **explicitly declare** they need access to sensitive resources get them. Others remain clean and safe.

---

## ✅ What You’re Learning to Do

* Build **composable, testable tools** by relying on dependency injection via `ActionContext`
* **Reduce cognitive load** on the agent orchestration loop
* Increase **security** and **modularity**
* Follow a **clean architecture** that scales





## ✅ 1. **What Exactly Is `ActionContext`?**

Think of `ActionContext` as a **backpack of resources** that the agent carries into each tool it uses.

It’s a lightweight object that:

* Contains **runtime-specific data** (like memory, LLM, auth tokens, feature flags, etc.)
* Lets each tool **request only what it needs**
* Enables **dependency injection** without hard-coding tools to specific global state or agent internals

### Example:

```python
context = ActionContext({
    "memory": memory,
    "llm": my_llm,
    "auth_token": "abc123"
})
```

Then in a tool:

```python
def update_profile(action_context: ActionContext, username: str) -> dict:
    token = action_context.get("auth_token")
    ...
```

This is cleaner and more flexible than importing globals, and allows tools to run in **test environments, different agents, or different contexts** without breaking.



## ✅ **What Is `ActionContext` Really?**

Yes — it's a **dictionary-like container** for everything an agent or tool might need **at runtime**, such as:

* `memory`
* `llm` model reference
* `auth_token`
* `agent_registry`
* `user_profile`
* `logger`, `db`, etc.

> Think of it as a **“request-level dependency bundle.”**

---

## 💡 Why Not Just Use Global Variables or a Requirements Doc?

Let’s compare:

| Concept              | Description                                                                  | Problems It Introduces                                                                     |
| -------------------- | ---------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------ |
| **Global Variables** | Shared state across your whole app (e.g., `AUTH_TOKEN = "abc123"`)           | - Hard to test<br>- Hard to override per request<br>- Risk of accidental reuse or mutation |
| **Requirements Doc** | High-level expectations of what a tool needs (e.g., doc says "needs memory") | - Not enforced<br>- Not machine-readable<br>- Easy to drift                                |
| **ActionContext**    | **Explicit, scoped, per-request object** with injected dependencies          | ✅ Easy to test, control, override, document, extend                                        |

---

## 🧠 Key Reasons Why `ActionContext` Is Preferred

### 1. **Dependency Injection Without Tight Coupling**

Instead of tools saying:

```python
from my_agent import memory
```

They say:

```python
def tool(action_context: ActionContext):
    memory = action_context.get("memory")
```

This allows tools to be:

* Reusable in other systems
* Tested in isolation
* Cleanly separated from the agent’s internals

---

### 2. **Per-Request Isolation**

Each **agent run**, **user session**, or **task** can have its own `ActionContext`. For example:

```python
agent.run(
    user_input="Review this code",
    action_context_props={"auth_token": "abc123", "env": "dev"}
)
```

No global collisions, no shared state bugs.

---

### 3. **Single Source of Truth, But Scoped**

Like you said: yes, it's a **single source of truth**, but with a key difference:

> It’s scoped to a single agent run — *not global to the whole app.*

This keeps things:

* Traceable (you know what was passed in this execution)
* Replaceable (swap in a different LLM, memory, etc.)
* Safer for multi-agent systems

---

### 4. **Pluggable Execution Environments**

You can plug in different things at runtime:

* Swap LLM from GPT-4 to Claude
* Swap memory backends (in-memory vs. Redis)
* Add a new service (logging, telemetry) without modifying tools

---

### ✅ Final Analogy

* **Global vars** are like public graffiti walls — anyone can write, anyone can break things.
* **Requirements docs** are like wish lists — not enforced.
* **ActionContext** is like a **backpack** each agent run carries — with only the tools and info it needs, packed cleanly and intentionally.





When you run this:

```python
agent.run(
    user_input="Review this code",
    action_context_props={"auth_token": "abc123", "env": "dev"}
)
```

You are doing **dynamic, per-run overrides** of the values that go into the `ActionContext`.

---

## 🔧 How It Works Internally

In the agent’s `run()` method (or orchestrator loop), something like this happens:

```python
action_context = ActionContext({
    'memory': memory,
    'llm': self.generate_response,   # Default injected dependencies
    **action_context_props           # 👈 This merges or overrides any defaults
})
```

This is equivalent to:

```python
# Resulting context dictionary:
{
    'memory': memory,
    'llm': self.generate_response,
    'auth_token': 'abc123',
    'env': 'dev'
}
```

So now, **inside any tool**, you can access those injected values:

```python
auth_token = action_context.get("auth_token")  # returns 'abc123'
env = action_context.get("env")                # returns 'dev'
```

---

## 🔁 Can This Override "Globals"?

Not exactly global in the usual Python sense — but:

* You might have *default config values* inside the agent itself.
* Passing values in `action_context_props` allows you to **override those defaults per run**.
* You don’t mutate any real global state — it’s all contained in that **one `ActionContext` instance per run**, which is safe and scoped.

---

## 🧠 Why This Matters

This gives you:

| Benefit                | Why It’s Useful                                       |
| ---------------------- | ----------------------------------------------------- |
| ✅ Scoped configuration | No risk of bleed-over between sessions or agents      |
| ✅ Runtime overrides    | You can change model, auth, memory, etc. on the fly   |
| ✅ Cleaner architecture | Tools don’t import or depend on outer systems         |
| ✅ Easier testing       | You can unit test a tool with fake memory or mock LLM |

---

## 🧪 TL;DR Recap

Yes — you're using `action_context_props={...}` to:

* **Inject dependencies** dynamically at runtime
* **Override defaults** set inside the agent
* Keep everything **isolated per execution**
* Avoid tight coupling or global variables






## 🧠 What does “Scoped Configuration” mean?

### 📌 Definition:

**Scoped configuration** means your settings or data:

* Only exist **within a specific scope**
* Are **isolated from the rest of the system**
* Are discarded or reset once that scope is done

In this context, the **scope is a single agent execution**.

---

### ✅ Why it matters:

| Without Scoped Config                               | With Scoped Config (like `ActionContext`) |
| --------------------------------------------------- | ----------------------------------------- |
| Global variables persist between runs, can conflict | Each run gets its own clean context       |
| Harder to debug & test                              | Easy to test and isolate                  |
| Risk of data leakage across agents/users            | Secure, modular, and predictable          |

---

### 🧼 Analogy:

Think of `ActionContext` like a **backstage clipboard** that’s handed to each actor (tool or agent) when they walk on stage. It contains only what they need **for that scene** — and is tossed out afterward.

No one shares their clipboard. No one peeks at the previous actor’s. That’s scoped configuration. ✨





✅ Correct Terminology

* **`ActionContext`**: A structured container that holds **dependencies** (yes, that's the correct term) and shared runtime data like memory, auth tokens, LLMs, etc.
* **`action_context_props`**: A **dictionary of overrides or additions** that get merged into the default `ActionContext` when the agent is run.

---

### ✅ Why This Pattern Is Great

#### 🧱 **Structured (Rigid)**

* Every tool and agent knows where to find its dependencies: `action_context.get("something")`
* Avoids global variables and hardcoded configs
* Makes testing and modularity cleaner

#### 🧩 **Flexible**

* You don’t have to rewrite tools or agents when needs change.
* You just **inject different props at runtime**:

  ```python
  agent.run(
      user_input="Do something secure",
      action_context_props={"auth_token": "xyz", "llm": test_llm}
  )
  ```
* This allows tools and agents to behave differently in **prod, dev, test, or even across users or requests**.

---

### 🛠 Real-World Analogy

Imagine:

* `ActionContext` is like a **backpack** the agent always carries.
* It comes with a few standard items: notebook (`memory`), map (`llm`), etc.
* But depending on the hike (agent run), you might throw in a snack (`auth_token`) or replace the map with a newer one (`llm` override).

---

### ✅ Summary

| Concept                | Purpose                                                    |
| ---------------------- | ---------------------------------------------------------- |
| `ActionContext`        | Base container for shared runtime state & dependencies     |
| `action_context_props` | Custom values merged at runtime to tailor agent behavior   |
| Benefit                | Combines strong structure **with** per-request flexibility |






### 🧪 What Is **Dependency Injection (DI)**?

**Dependency Injection** is a *software design pattern* where you **don't hardcode dependencies** inside your function, class, or tool. Instead, you **inject them from the outside** — making your code more flexible, modular, and testable.

---

### 🔄 Traditional (Hardcoded) vs Dependency Injection

#### ❌ Hardcoded (Tightly Coupled):

```python
def analyze_code():
    llm = OpenAI(api_key="secret")  # hardcoded dependency
    return llm.generate("review this code")
```

* You can't test it easily with a fake LLM.
* You can't swap models or keys without rewriting code.

---

#### ✅ Dependency Injected:

```python
def analyze_code(action_context: ActionContext):
    llm = action_context.get("llm")  # injected
    return llm.generate("review this code")
```

* You can inject any LLM (e.g., mock for testing, OpenAI, Anthropic).
* All dependencies are **externalized** — not buried inside logic.

---

### 💼 In Your Case

In your system, `ActionContext` is the mechanism for **injecting dependencies** like:

* `llm`
* `auth_token`
* `memory`
* `agent_registry`
* `environment` (e.g., dev/prod)

When you pass in:

```python
agent.run(..., action_context_props={"auth_token": "xyz", "llm": mock_llm})
```

You’re performing **true dependency injection**, but in a way that’s:

* Lightweight
* Runtime-configurable
* Cleanly scoped to just that agent run

---

### 📌 Summary

| Term              | Meaning                                                                   |
| ----------------- | ------------------------------------------------------------------------- |
| **Dependency**    | Something your code relies on (like an LLM, token, database, memory)      |
| **Injection**     | Providing that dependency from the outside, instead of creating it inside |
| **ActionContext** | Your container for injected dependencies                                  |

This pattern is widely used in professional software engineering — especially in frameworks like:

* **Spring** (Java)
* **Django** or **FastAPI** (Python)
* **React/Angular** (JS)

You’re applying the same architectural discipline to **AI agents** 👏


